# **Time series analysis** on Istanbul mean temperature data set
# (İstanbul ortalama sıcaklık veri setinde **zaman serisi analizi**)

**+--- NOTE ---+**
<br> **datetime** fonksiyonunun **Keyword**'lerini, GOOGLE'ye **"pandas datetime ofsets"** veya **"pandas time series / date functionality"** yazarak veya aşağıdaki linkten bulabiliriz **;**
<br> https://pandas.pydata.org/docs/user_guide/timeseries.html
<br> Bu linke tıklayıp biraz aşağılara inerek TÜM kullanım şekillerini bulabiliriz...

Veri Setini indirdiğimiz web sitesi **:**
<br> https://www.ecad.eu/
<br> Veri Setimizi açarak, veriyi nasıl okumamız gerektiği ile ilgili bilgileri, ilk satırlarda öğrenebiliriz...

+ **Veri Setimizdeki Kolonlar :**
+ + **DATE ;**
+ + + Yıl **-** Ay **-** Gün
+ + **TG ;**
+ + + Kolonda yazan değerleri, **'0.1'** ile çarparak gerçek hava sıcaklığı derecesine ulaşıyoruz.
+ + **Q.TG ;**
+ + + **'0' =** Sağlam veri demek.
+ + + **'1' =** Şüpheli veri demek.
+ + + **'9' =** Eksik veri demek.

In [1]:
# import pandas library as pd :

import pandas as pd

+ First,read **"istanbul_temp.txt"** file into a **csv** file **:**
<br> İlk önce **"istanbul_temp.txt"** dosyasını bir **csv** dosyasına okuyun **:**
+ Normalde Pandas, Veri Setindeki ilk satırı kolon satırı olarak görür... Ancak Veri Setimizin ilk satırlarında açıklamalar yazdığı için, bizim bu durumu iptal edip yerine yeniden kolon isimlerini belirlememiz lazım. Yani başlığı iptal etmek için **"header=None"** komutunu, yeni başlık eklemek için de **"names="** komutunu kullanmamız lazım **:**
+ Veri Setimizin İlk **22** satırında açıklama olduğu için **,** bu satırları atlamak istiyoruz... Bunun için de **"(skiprows=___)"** komutunu kullanacağız **:**

In [ ]:
#read data into a data frame :

data=pd.read_csv("istanbul_temp.txt", skiprows=22, header=None, names=["STAID","SOUID","DATE","TG","Q_TG"])

+ Veya; Veri Setini incelediysek, ilk **22** satırı değil de ilk **21** satırı atlarsak zaten direkt başlığın yazılı olduğu satırı otomatik olarak pandas okuyacaktı **:**

In [2]:
#   OR

data=pd.read_csv("istanbul_temp.txt", skiprows=21)

In [3]:
data.columns

Index([' STAID', ' SOUID', '    DATE', '   TG', ' Q_TG'], dtype='object')

+ Yukarıda gördüğümüz gibi Veri Setindeki başlığı kullanarak okuma işlemi yaparsak, kolon isimlerinde bir problem ile karşılaşıyoruz. Kolon isimlerinin başında gereksiz boşluklar görüyoruz ve bu analiz işlemi yaparken bizim işimizi zorlaştırıyor... Bunu çok kolay bir şekilde düzeltebiliriz. Boşluk karakterlerini kaldırarak bu sorunu çözebiliriz **:**

In [4]:
data.columns = data.columns.str.strip();
data.columns

Index(['STAID', 'SOUID', 'DATE', 'TG', 'Q_TG'], dtype='object')

Yukarıda gördüğümüz gibi, bunu çok kolay bir şekilde düzeltebildik...

+ Check top of the dataframe **:**
<br> Neler olduğunu görmek için, veri çerçevesinin EN ÜSTÜNÜ ( .head ) kontrol et **:**

In [5]:
data.head()

,STAID,SOUID,DATE,TG,Q_TG
0,248,101231,19290101,126,0
1,248,101231,19290102,133,0
2,248,101231,19290103,131,0
3,248,101231,19290104,118,0
4,248,101231,19290105,106,0


+ Check bottom of the dataframe to see what is happening **:**
<br> Neler olduğunu görmek için, veri çerçevesinin EN ALTINI ( .tail ) kontrol edin **:**

In [6]:
data.tail()

,STAID,SOUID,DATE,TG,Q_TG
32410,248,101231,20170926,-9999,9
32411,248,101231,20170927,-9999,9
32412,248,101231,20170928,-9999,9
32413,248,101231,20170929,-9999,9
32414,248,101231,20170930,-9999,9


Yukarıda gördüğümüz gibi, son 5 satırdaki veriler hep **Eksik Veri...**

+ Note that in the file it is written as “9” in Q_TG columns shows the missing data.
<br> 30-34 Q_TG : Quality code for RR (0='valid'; 1='suspect'; 9='missing')
<br> Drop the rows that have 9’s in the Q_TG column.
<br> data=data[data["Q_TG"]!=9] Generates a new dataframe where rows that have “9” omitted.
+ **Q_TG** kolonunun değeri **9** olan, yani eksik veri olan satırları veri setimizden silelim... Yani tablomuzun **Q_TG** kolonunda **9** değeri olmayan satırları seçelim ve veri setimize onları atayalım **:**

In [7]:
data = data[data.Q_TG != 9];

In [8]:
#   Now check the bottom of the file :
#   Şimdi dosyanın en altını kontrol edin :

data.tail()

,STAID,SOUID,DATE,TG,Q_TG
27419,248,101231,20040127,36,0
27420,248,101231,20040128,85,0
27421,248,101231,20040129,128,0
27422,248,101231,20040130,78,0
27423,248,101231,20040131,46,0


+ To make sure check unique elements on Q_TG column **:**
<br> Q_TG sütunundaki benzersiz öğeleri kontrol ettiğinizden emin olmak için **:**
<br> Benzersiz (unique) verilere de bakalım ve tam olarak emin olalım **:**

In [10]:
data.Q_TG.unique()

array([0, 1], dtype=int64)

In [11]:
data.Q_TG.nunique()

2

Yukarıda gördüğümüz gibi, **Q_TG** kolonunda sadece 2 farklı veri var ve bu veriler **"0" ve "1".**

+ Şimdi de, kaç tane veri **"0"** yani sağlam, kaç tane veri **"1"** yani şüpheli onu görelim **:**

In [12]:
data.Q_TG.value_counts()

0    27416
1        8
Name: Q_TG, dtype: int64

In [13]:
#   OR

data.value_counts("Q_TG")

Q_TG
0    27416
1        8
dtype: int64

Yukarıda gördüğümüz gibi, tüm veri setimizde sadece 8 tane veri **"1"**, yani şüpheli veri.

+ Şimdi de **TG** kolonundaki değerleri **0.1** ile çarparak, gerçek sıcaklık değerlerine çevirelim **:**

In [9]:
data.TG = data.TG * 0.1;
data.head()

,STAID,SOUID,DATE,TG,Q_TG
0,248,101231,19290101,12.6,0
1,248,101231,19290102,13.3,0
2,248,101231,19290103,13.1,0
3,248,101231,19290104,11.8,0
4,248,101231,19290105,10.6,0


+ Şimdi asıl ilgilendiğimiz bölüme gelelim...
<br> Öncelikle verileri nasıl tuttuğumuza bakalım **:**

In [92]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27424 entries, 0 to 27423
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   STAID   27424 non-null  int64  
 1   SOUID   27424 non-null  int64  
 2   DATE    27424 non-null  int64  
 3   TG      27424 non-null  float64
 4   Q_TG    27424 non-null  int64  
dtypes: float64(1), int64(4)
memory usage: 1.3 MB


Yukarıda gördüğümüz gibi, mesela STAID kolonunda 27424 adet NaN OLMAYAN veri varmış ve tipi de **int64**'müş.
+ DATE kolonunun da tipi **int64** imiş. Bizim bunu **"datetime"** hale çevirmemiz gerekiyor... Bunu en başta, daha Veri Setini okurken de yapabilirdik... Ama şu an bu şekilde yapmak istiyoruz **:**

+ DATE column now is in string format we should change it into a datetime object format **:**
<br> DATE sütunu **int64** biçiminde, bu sütunu **datetime** biçimine değiştirmeliyiz **:**
<br> Use **pd.to_datetime()** function **:**

In [15]:
data.DATE.head()

0    19290101
1    19290102
2    19290103
3    19290104
4    19290105
Name: DATE, dtype: int64

In [16]:
data.DATE = pd.to_datetime(data.DATE)

+ Check the top of DATE column **:**
<br> DATE sütununun en üstünü kontrol edin **:**

In [17]:
data.DATE.head()

0   1970-01-01 00:00:00.019290101
1   1970-01-01 00:00:00.019290102
2   1970-01-01 00:00:00.019290103
3   1970-01-01 00:00:00.019290104
4   1970-01-01 00:00:00.019290105
Name: DATE, dtype: datetime64[ns]

Yukarıda gördüğümüz gibi, **datetime** hale dönüştürdük...

**++++ CRITICAL NOTE ++++**
<br> **ÖNEMLİ !**
<br> **Tarih - Zaman** kodları yazarken bazen hata almayız ama bir şekilde sonuçlar garip çıkabilir... **String , Integer , float , vb.** olan değerleri **datetime** hale çevirdiğimizde hata almasak da **,** farklı şekillerde okunabildiği için garip sonuçlar alabiliriz... Bunu çözmek için de **, pd.to_datetime( )** fonksiyonuna gelip SHIFT + TAB tuşlarına bastığımızda **, (format="...")** diye bir komut görürüz... Bu komutu yazarak işlem yaparsak daha garanti bir sonuç alırız **!**
<br> **Formata da ;** Veri Setimizi açıp **, Tarih-Saatin** nasıl yazıldığına bakarak karar veririz... Aralardaki **noktalama işaretleri bile Format belirlemede** çok önemlidir **!... :**

In [10]:
data.DATE = pd.to_datetime(data.DATE, format="%Y%m%d")

In [11]:
data.DATE.head()

0   1929-01-01
1   1929-01-02
2   1929-01-03
3   1929-01-04
4   1929-01-05
Name: DATE, dtype: datetime64[ns]

Böyle yukarıdaki gibi yaparsak bir hata alma ihtimalimiz kalmıyor **!**
<br> **+--- NOTE --->** Bu defa, saat-dakika-saniye...... ve devamının görünmemesinin sebebi de, formatta sadece Yıl-Ay-Gün görmek istediğimizi belirtmemizdendir...
<br> **+--- NOTE --->** Yukarıda gördüğümüz, **'datetime64[ns]'** ifadesindeki **[ns]** de, **nanosaniye**'ye kadar çalışabilirsiniz demek... Yani **datetime,** nanosaniye cinsine kadar çalışmamıza izin veriyor...

You see that data type is changed to **datetime64**.
<br> Yukarıdaki OUT'da, veri türünün **datetime64** olarak değiştirildiğini görüyorsunuz.

+ Set the index of the dataframe as the **DATE** column **:**
<br> For this use "**dataframe.set_index( )**" function **:**
+ Veri çerçevesinin index'ini (dizinini), **DATE** sütunu olarak ayarlayın **:**
<br> Yani index'imiz **Zaman Index**'i olsun... Yani index'imiz 0, 1, 2 değil de, zaman-tarih olsun...
<br> Yani kolonda bulunan **DATE** kolonunu, **index**'e atamamız gerekiyor **:**
<br> Bunun için "**dataframe.set_index( )**" fonksiyonunu kullanın **:**

In [12]:
data.set_index("DATE", inplace=True)

+ Again check the top of dataframe **:**
<br> Veri çerçevesinin en üstünü tekrar kontrol edin **:**

In [13]:
data.head()

,STAID,SOUID,TG,Q_TG
DATE,,,,
1929-01-01,248,101231,12.6,0
1929-01-02,248,101231,13.3,0
1929-01-03,248,101231,13.1,0
1929-01-04,248,101231,11.8,0
1929-01-05,248,101231,10.6,0


+ **Yaptığımız işlemler sırası ile:**
+ + Öncelikle Veri Setimize uygun bir şekilde **csv** dosyasına **okuyoruz**,
+ + Sonra oluşturduğumuz tablonun başlıklarını inceleyip sorun varsa düzeltiyoruz,
+ + Sonra güvenilmez ve eksik verileri, veri setimizden uzaklaştırıyoruz,
+ + Sonra kolonlarımızda düzeltilmesi gereken yerler varsa düzeltiyoruz,
+ + Daha sonra; String, Integer, Float vb. tipte olan tarih verilerini **datetime** tipine çeviriyoruz,
+ + Ve daha sonra da bu **datetime** tipine çevirdiğimiz tarih kolonunu, **index**'e atıyoruz.

Yukarıdaki işlemleri tamamladıktan sonra artık analize hazır hale geliyoruz...

## Resampling Examples (Resample Örnekleri)

+ We have daily mean temperatures, calculate the yearly average temperatures **:**
<br> Use **dataframe.resample( )** function for this create a new dataframe "**data_yearly**" **:**
+ İstanbul'un günlük ortalama sıcaklıklarına sahibiz... **Yıllık ortalama** sıcaklıkları hesaplayın **:**
<br> Bunun için **dataframe.resample( )** fonksiyonunu kullanın ve yeni bir **"data_yearly"** veri çerçevesi oluşturun **:**

In [34]:
data.head()

,STAID,SOUID,TG,Q_TG
DATE,,,,
1929-01-01,248,101231,12.6,0
1929-01-02,248,101231,13.3,0
1929-01-03,248,101231,13.1,0
1929-01-04,248,101231,11.8,0
1929-01-05,248,101231,10.6,0


**+--- NOTE ---> Yıllık** olarak hesaplamak için kullanılan frekans komutu **=** **'A'** veya **'AS'**'dir **!**

In [14]:
data_yearly = data.resample("A").mean();
data_yearly.head()

,STAID,SOUID,TG,Q_TG
DATE,,,,
1929-12-31,248.0,101231.0,13.904110,0.008219
1930-12-31,248.0,101231.0,14.788493,0.000000
1931-12-31,248.0,101231.0,14.329041,0.000000
1932-12-31,248.0,101231.0,15.054372,0.000000
1933-12-31,248.0,101231.0,13.490685,0.000000


+ Yukarıda fark ettiysek **;** özel bir kolon seçmediğimiz için, tüm kolonların ortalamasını aldı... Şimdi de sadece **TG** kolonunun ortalamasını gösterelim **:**

In [78]:
data_yearly.TG = data.resample("A").TG.mean();
data_yearly.TG.tail()

DATE
2000-12-31    15.195902
2001-12-31    15.867397
2002-12-31    15.420822
2003-12-31    14.674795
2004-12-31     5.254839
Freq: A-DEC, Name: TG, dtype: float64

+ Şimdi de 5 yıllık ortalama sıcaklıkları bulalım **:**

In [15]:
data_five_year = data.resample("5AS").mean();
data_five_year.head()

,STAID,SOUID,TG,Q_TG
DATE,,,,
1929-01-01,248.0,101231.0,14.313746,0.001643
1934-01-01,248.0,101231.0,14.676616,0.000000
1939-01-01,248.0,101231.0,13.719168,0.001095
1944-01-01,248.0,101231.0,14.276519,0.000000
1949-01-01,248.0,101231.0,14.188554,0.000548


---
---

---------->

+ Now **plot** these yearly averages **:**
<br> Şimdi bu yıllık ortalamayı grafikte gösterelim **:**

In [16]:
data_yearly.TG.plot()

<AxesSubplot:xlabel='DATE'>

Yukarıdaki OUT'da bir grafik oluşması gerekiyordu ama olmadı... Bu sorunu ileride çözeceğim...

<----------

---
---

Note that something looks weird around 2000, we have a sudden drop!
<br> 2000 yılı civarında bir şeylerin tuhaf göründüğünü unutmayın, ani bir düşüş yaşıyoruz!

In [80]:
data_yearly.tail(3)

,STAID,SOUID,TG,Q_TG
DATE,,,,
2002-12-31,248.0,101231.0,15.420822,0.0
2003-12-31,248.0,101231.0,14.674795,0.0
2004-12-31,248.0,101231.0,5.254839,0.0


**+--- NOTE --->** Biz yukarıda ortalamaları aldık ama hangi yılda kaç eksik veri var kaç sorunlu veri var bakmadık... Mesela bazı yıllarda 300 gün için veri varken bazı yıllarda 5 gün için veri olabilir... Bunlara dikkat etmeliyiz...

+ At the year 2004, tempearature is very small compared to other years...
<br> To investigate this, find how many **datapoints** we have in **"data"** dataframe **:**
<br> You can use **size** function to create a series called **"yearly_size" :**
+ 2004 yılında sıcaklık diğer yıllara göre çok düşük görünüyor...
<br> Bunu araştırmak için **; "data"** veri çerçevesinde **,** kaç tane **veri noktasına** sahip olduğumuzu bulalım. Yani hangi yıllarda kaç tane verimiz var bunu bulalım **:**
<br> **"yearly_size"** adlı bir SERİ (DİZİ) oluşturalım. **".size( )" , "count( )"** gibi fonksiyonları kullanabilirsiniz **:**

In [17]:
yearly_size = data.resample("A").size() ;
yearly_size.tail()

DATE
2000-12-31    366
2001-12-31    365
2002-12-31    365
2003-12-31    365
2004-12-31     31
Freq: A-DEC, dtype: int64

Yukarıda, "**.resample( )**" fonksiyonu ile yıllık olarak GRUPLADIK... "**.size( )**" fonkisiyonu ile de HER GRUP içerisindeki ELEMAN SAYISINI öğrendik **!**

We got the answer **!!**
<br> the year 2004 has only 31 datapoints, probably data from january that is why the aveage temperature is so low **!**
<br> Cevabı aldık **!!**
<br> 2004 yılı sadece 31 veri noktasına sahip. Muhtemelen de bu veriler bir kış ayına ait. Hatta sanırım Ocak ayına ait. Bu yüzden ortalama sıcaklık çok düşük görünüyor **!**

+ Let's check for which years the number of days is not 365
<br> Type **yearly_size!=365**. What sort of output do we have **?**
+ Hangi yıllar için gün sayısının 365 olmadığını kontrol edelim **:**
<br> **yearly_size!=365** yazalım. Ne tür bir çıktımız var **?**
+ Yani şimdi de 365 veriye eşit olmayan yıllar hangi yıllar onu öğrenmek istiyoruz **:**

In [18]:
yearly_size != 365

DATE
1929-12-31    False
1930-12-31    False
1931-12-31    False
1932-12-31     True
1933-12-31    False
              ...  
2000-12-31     True
2001-12-31    False
2002-12-31    False
2003-12-31    False
2004-12-31     True
Freq: A-DEC, Length: 76, dtype: bool

+ We got a **boolean series**, if we insert this series into **yearly_size** series, we will get rows that are **True:**
<br> Yukarıda bir **boolean** serimiz var, bu seriyi **"yearly_size"** serisine eklersek **True** olan satırlar elde ederiz.
<br> Yani bu yukarıdaki sorguyu **index** olarak kullanırsak bize direkt sonucu verir **:**

In [19]:
yearly_size[yearly_size != 365]

DATE
1932-12-31    366
1936-12-31    366
1940-12-31    366
1944-12-31    366
1948-12-31    366
1952-12-31    366
1956-12-31    366
1960-12-31    366
1964-12-31    366
1968-12-31    366
1972-12-31    366
1976-12-31    366
1980-12-31    366
1984-12-31    366
1988-12-31    366
1992-12-31    366
1996-12-31    366
2000-12-31    366
2004-12-31     31
dtype: int64

All the rows except of the last one has 366.
<br> So this years are leap years.
<br> Yukarıda, sonuncusu hariç tüm satırlarda 366 var.
<br> Yani bu yıllar artık yıllar. Çünkü her 4 yılda bir artık yıl olur...

+ Accesing data for certain years **;**
<br> Let's get the yearly_size information only for the years between 1950 and 1960 **:**
<br> Try first yearly_size[1950:1960], see what happens **:**
+ Belirli yıllara ait verilere erişim için **;**
<br> Sadece 1950 ile 1960 arasındaki yıllar için **yıllık_size** bilgisini alalım **:**
<br> İlk önce **yearly_size[1950:1960]** komutunu deneyin, ne olduğunu görün **:**
+ Yani elimizde bir adet zaman serisi var. Ve bu seri içerisinde sadece 1950 ve 1960 arasındaki verileri bulmak istiyoruz **:** 

In [20]:
yearly_size[1950:1960]

Series([], Freq: A-DEC, dtype: int64)

Yukarıdaki gibi yaptığımızda, gördüğümüz gibi çalışmıyor... Çünkü burada bir zaman yok, yani bir **datetime** index'i yok. Biz burada tamsayı aratıyoruz ve olmuyor...
<br> Yapmamız gereken, eğer içinde arama yapmaya çalıştığımız seri bir **datetime** serisi ise ona göre biz de o şekilde aratmalıyız... **:**

+ We didn't get anything because our index is a **datetime index** and we should enquiry with the same data structure
<br> We write the date either in **string** format **:**
<br> yearly_size["1/1/1950":"1/1/1960"]
+ Peki nasıl aratabiliriz **?** Normal bir şekilde tam tarihi yazar gibi yazarsak aratabiliriz **:**
<br> Tarihi, **string** formatında yazıyoruz:
<br> yearly_size["1/1/1950":"1/1/1960"]

In [21]:
yearly_size["1/1/1950" : "1/1/1960"]

DATE
1950-12-31    365
1951-12-31    365
1952-12-31    366
1953-12-31    365
1954-12-31    365
1955-12-31    365
1956-12-31    366
1957-12-31    365
1958-12-31    365
1959-12-31    365
Freq: A-DEC, dtype: int64

+ **Or**
<br> We can create a **datetime** object for this **:**
<br> First, **import datetime** object and create **timestamps** for 1950 and 1960 **:**
+ **Veya**
<br> Bunun için bir **datetime** nesnesi oluşturabiliriz **:**
<br> İlk olarak, **datetime** nesnesini **içe aktarın** ve 1950 ve 1960 için **zaman damgaları** oluşturun **:**
+ Yani biz kendimiz manuel olarak **datetime** oluşturabiliriz **:**
<br> Ve aslında en düzgün aratma şekli de budur **:**

In [22]:
from datetime import datetime

In [23]:
t1 = datetime(1950,1,1)

In [24]:
t1

datetime.datetime(1950, 1, 1, 0, 0)

**+--- NOTE --->** Yukarıda gördüğümüz gibi, parantez içerisine EN AZ; Yıl, Ay ve Gün'ü yazmak zorundayız **!**
<br> Saat, dakika, saniye, salise, ... ve daha nicelerini de yazabiliriz ama ilk 3'ü zorunlu **!**

In [25]:
type(t1)

datetime.datetime

+ We can put these **datetime** objects as indices for **time series** **:**
<br> Bu **datetime** nesnelerini **zaman serileri** için indeksler olarak koyabiliriz **:**
+ Yani elimizde bir **zaman serisi** varsa, ve burada bir **slice (dilimleme, parçalama)** yapmak istiyosak, şu şekilde yapabiliriz **:**

In [26]:
yearly_size[datetime(1950,1,1): datetime(1960,1,1)]

DATE
1950-12-31    365
1951-12-31    365
1952-12-31    366
1953-12-31    365
1954-12-31    365
1955-12-31    365
1956-12-31    366
1957-12-31    365
1958-12-31    365
1959-12-31    365
Freq: A-DEC, dtype: int64

En garanti yöntem de bu yukarıda uyguladığımız yöntemdir...

### Plot average temperatures in March over the years
### (Tüm yılların Mart aylarındaki ortalama sıcaklıklarını grafikte gösterin)

+ First, get montly averages **:**
<br> Create a **time series** for monthy averages, name this series **"montly" :**
+ İlk olarak, aylık ortalamaları alın **:**
<br> Aylık ortalamalar için bir **zaman serisi** oluşturun, bu seriyi **"montly"** olarak adlandırın **:**

In [27]:
monthly = data.resample('M').mean() ;
monthly.head()

,STAID,SOUID,TG,Q_TG
DATE,,,,
1929-01-31,248.0,101231.0,5.454839,0.000000
1929-02-28,248.0,101231.0,1.957143,0.071429
1929-03-31,248.0,101231.0,3.445161,0.032258
1929-04-30,248.0,101231.0,9.976667,0.000000
1929-05-31,248.0,101231.0,18.883871,0.000000


+ We got the averages for each month.
<br> But we are interested in only March months **:**
+ Her ay için ortalamaları aldık.
<br> Ama biz sadece Mart aylarıyla ilgileniyoruz **:**

+ Since our index is a **datetime** object, one good feature of this object is that we can get such information like month,day,hour etc.
<br> Try **:** monthly.index.month
+ İndeksimiz bir **datetime** nesnesi olduğundan, bu nesnenin iyi bir özelliği de; ay, gün, saat gibi bilgileri alabilmemizdir.
<br> Deneyin **:** aylık.index.ay
+ Şimdi, **'monthly'** dediğimiz şey aynı zamanda bir **DataFrame**'dir. Bunun **index**'ine rahatlıkla erişebiliriz. Yani **.index( )** fonksiyonunu çalıştırdığımız zaman bize **index** nesnelerini döner... Burada da **index**'lerimiz **datetime** olduğu için, **'monthly'** isimli zaman serisi atamamızın, **attributes (özellikler)**'ine rahatlıkla erişebiliriz... **:** 

In [28]:
monthly.index.month

Int64Index([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10,
            ...
             4,  5,  6,  7,  8,  9, 10, 11, 12,  1],
           dtype='int64', name='DATE', length=901)

+ Veya rakam olarak değil de direkt ayların isimlerini görmek istiyorsak da **'.month_name()' metodunu** kullanabiliriz **:**
<br> Ama dediğimiz gibi **'.month_name( )'** bir **attribute** değil bir **metotdur !**
<br> Çünkü gördüğümüz gibi sonunda **parantez ( )** vardır...

In [29]:
monthly.index.month_name()

Index(['January', 'February', 'March', 'April', 'May', 'June', 'July',
       'August', 'September', 'October',
       ...
       'April', 'May', 'June', 'July', 'August', 'September', 'October',
       'November', 'December', 'January'],
      dtype='object', name='DATE', length=901)

+ Here 1 is for jan, 2 is for feb and so on.
<br> If we select rows whose **monthly.index.month** is equal to 3, then we select March months **:**
<br> We create a new series for Marches **:**
+ Burada, 1: Ocak, 2: Şubat ... vb.'dir.
<br> **monthly.index.month** '3' olan satırları seçersek, Mart aylarını seçeriz **:**
<br> Mart ayları için yeni bir seri oluşturuyoruz **:**
<br> Ve bunu, **boolean** olarak değil de **index** olarak kullanalım / görelim **:**

In [30]:
monthy_march = monthly[monthly.index.month == 3] ;
monthy_march.head()

,STAID,SOUID,TG,Q_TG
DATE,,,,
1929-03-31,248.0,101231.0,3.445161,0.032258
1930-03-31,248.0,101231.0,9.393548,0.000000
1931-03-31,248.0,101231.0,7.825806,0.000000
1932-03-31,248.0,101231.0,6.216129,0.000000
1933-03-31,248.0,101231.0,6.719355,0.000000


+ Let's plot this average temperatures **:**
+ Bu ortalama sıcaklıkları grafikte gösterelim (çizelim) **:**

In [31]:
monthy_march.TG.plot()

<AxesSubplot:xlabel='DATE'>

Yukarıda gördüğümüz gibi, yine grafik olarak bir OUT göremedik... Bu sorunu çözeceğiz...